In [22]:
import pandas as pd
import numpy as np


train_file = 'train.json'
with open(train_file, 'r') as f:
        df=pd.read_json(f)
        
df=df.T
features=df["features"].values
valence = df["valence"].values
activation = df["activation"].values

all_lengths = [len(seq) for seq in features]
max_length = max(all_lengths)
x1 = [x for x in all_lengths if x > 300 and x <= 500]
x2 = [x for x in all_lengths if x > 500]
x3 = [x for x in all_lengths if x > 00]
print(len(x1), len(x2),len(x3))

average_length = np.mean(all_lengths)
print(average_length,max_length)


1311 461 7800
223.3270512820513 1707


In [23]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

def load_data(train_file):
    with open(train_file, 'r') as f:
         df=pd.read_json(f)
    df=df.T
    valence = df["valence"].values
    activation = df["activation"].values
    features = df["features"].values
    lengths = [len(seq) for seq in features]
    desired_seq_length = 500
    padded_x = np.zeros((len(features), desired_seq_length, len(features[0][0])))
    # loop through each sequence in x
    for i, seq in enumerate(features):
    # check if sequence length is greater than desired length
         if len(seq) > desired_seq_length:
            padded_x[i] = seq[:desired_seq_length]
         else:
            padded_x[i, :len(seq), :] = seq
    # sequence length is already equal to desired length

    train_data_X = torch.from_numpy(padded_x).float()

    # Define the label dictionary
    label_dict = {(1, 1): 'joy', (1, 0): 'pleasure', (0, 0): 'sadness', (0, 1): 'anger'}
    # Define a function to map valence and activation to label
    def map_label(row):
        return label_dict[(row['valence'], row['activation'])]
    # Add a new column with the label for each row
    df['label'] = df.apply(map_label, axis=1)
    # Drop the valence and activation columns
    df.drop(['valence', 'activation'], axis=1, inplace=True)
    # create a dictionary to map the emotion labels to numbers
    emotion_dict = {'joy': 0, 'pleasure': 1, 'sadness': 2, 'anger': 3}
    # replace the emotion labels with numbers in the dataframe
    df['label'] = df['label'].replace(emotion_dict)

    labels = df["label"].values
    train_data_y = torch.from_numpy(labels)

    return train_data_X,  train_data_y



In [24]:
import pandas as pd
import numpy as np
import torch

dev_file = 'dev.json'
with open(dev_file, 'r') as f:
         df=pd.read_json(f)
df=df.T
features = df["features"].values
lengths = [len(seq) for seq in features]
desired_seq_length = 500
padded_x = np.zeros((len(features), desired_seq_length, len(features[0][0])))
# loop through each sequence in x
for i, seq in enumerate(features):
    # check if sequence length is greater than desired length
      if len(seq) > desired_seq_length:
          padded_x[i] = seq[:desired_seq_length]
      else:
          padded_x[i, :len(seq), :] = seq
    # sequence length is already equal to desired length

dev_data_X = torch.from_numpy(padded_x).float()


In [25]:
# relatively complicated version with three convs and two lstms
import torch
import torch.nn as nn

class MyCnnModel(nn.Module):
    def __init__(self):
        super(MyCnnModel, self).__init__()
         # Defining a 2D convolution layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2))
            # Defining another 2D convolution layer
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.lstm1 = nn.LSTM(input_size=64, hidden_size=64, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=64, batch_first=True)
        self.fc1 = nn.Linear(60*64, 32)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(32, 4)

    def forward(self, x):
        #print(x.shape)  # print input shape
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #print("end of CNN", x.shape)  # print after avgpool3 layer
        x=x.squeeze(-1)
        x=x.transpose(1,2)
        x, _ = self.lstm1(x)
        #print(x.shape) 
        x,_ = self.lstm2(x) 
        #print("after lstm", x.shape) # print after LSTM layer
        x = x.reshape(x.size(0), -1)
        #print(x.shape) # print after LSTM layer
        x = self.fc1(x)
        #print(x.shape)  # print after classifier
        x = self.activation(x)
        x = self.fc2(x)
        #print(x.shape)  # print after activation
        return x


model = MyCnnModel()

In [6]:
train_file = 'train.json'

train_data_X,  train_data_y = load_data(train_file)


import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import random

train_data = Data.TensorDataset(train_data_X, train_data_y)   # merge the features x data and label y into one dataset

# Define the data loaders
train_loader = Data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
# print(len(train_loader)) = 244


In [26]:
model = MyCnnModel()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    


In [27]:
import torch
import torch.nn as nn
import copy
import time
from torch.optim import Adam

def train_model(model, train_loader, train_rate, criterion, optimizer, num_epochs):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    
    best_acc = 0.0
  
    batch_num =len(train_loader)
    train_batch_num = round(batch_num* train_rate)

    train_loss_all = []
    train_acc_all = []
    val_loss_all = []
    val_acc_all = []


    for epoch in range(num_epochs):
         print("Epoch {}/{}".format(epoch, num_epochs))
         print("-"*10)
        
       # there are two stages for each epoch
         train_loss = 0.0
         train_corrects = 0
         train_num =0
         val_loss = 0.0
         val_corrects = 0.0
         val_num = 0
       
         for step, (b_x, b_y) in enumerate(train_loader):
              b_x = b_x.to(device)
              b_y = b_y.to(device)

              if  step < train_batch_num:
                  
                model.train()

                output= model(b_x)
                pre_lab = torch.argmax(output, dim=1)
                loss = criterion(output, b_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * b_x.size(0)
                train_corrects += torch.sum(pre_lab == b_y.data)
                train_num += b_x.size(0)
                
              else:
                
                model.eval()
                output= model(b_x)
                pre_lab = torch.argmax(output, dim=1)
                loss = criterion(output, b_y)
                val_loss += loss.item() * b_x.size(0)
                
                val_corrects += torch.sum(pre_lab == b_y.data)
                val_num += b_x.size(0)
                
          
         train_loss_all.append(train_loss/train_num)
         train_acc_all.append(train_corrects.double().item()/train_num)
         val_loss_all.append(val_loss/val_num)
         val_acc_all.append(val_corrects.double().item()/val_num)

         print('{} Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch, train_loss_all[-1], train_acc_all[-1]))
         print('{} Val Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch, val_loss_all[-1], val_acc_all[-1]))

        
        # store the best model with paremeters         # Keep track of the best model weights
         if val_acc_all[-1]>best_acc:
                       best_acc = val_acc_all[-1]
                       best_model_wts = copy.deepcopy(model.state_dict())
          
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model
  

In [28]:
# Initialize the model, loss function, and optimizer

model = MyCnnModel()

#optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()


model= train_model(model, train_loader, 0.85, criterion, optimizer,num_epochs=120)

Epoch 0/120
----------
0 Train Loss: 1.1707, Train Acc: 0.4503
0 Val Loss: 1.3029, Val Acc: 0.3129
Epoch 1/120
----------
1 Train Loss: 1.1266, Train Acc: 0.4758
1 Val Loss: 1.0731, Val Acc: 0.5145
Epoch 2/120
----------
2 Train Loss: 1.0988, Train Acc: 0.4899
2 Val Loss: 1.2336, Val Acc: 0.3588
Epoch 3/120
----------
3 Train Loss: 1.0838, Train Acc: 0.5051
3 Val Loss: 1.1572, Val Acc: 0.4685
Epoch 4/120
----------
4 Train Loss: 1.0821, Train Acc: 0.5095
4 Val Loss: 1.1394, Val Acc: 0.4932
Epoch 5/120
----------
5 Train Loss: 1.0655, Train Acc: 0.5168
5 Val Loss: 1.1844, Val Acc: 0.4787
Epoch 6/120
----------
6 Train Loss: 1.0594, Train Acc: 0.5228
6 Val Loss: 1.3855, Val Acc: 0.3844
Epoch 7/120
----------
7 Train Loss: 1.0451, Train Acc: 0.5346
7 Val Loss: 1.0743, Val Acc: 0.4864
Epoch 8/120
----------
8 Train Loss: 1.0426, Train Acc: 0.5386
8 Val Loss: 1.0708, Val Acc: 0.5272
Epoch 9/120
----------
9 Train Loss: 1.0269, Train Acc: 0.5362
9 Val Loss: 1.2363, Val Acc: 0.4838
Epoch 10/1

In [29]:
#save the best model weights to a file
#torch.save(model.state_dict(),"speechemotion")


# load the model
model = MyCnnModel()

#model.load_state_dict(torch.load('speechemotion'))

# Set the model to evaluation mode
model.eval()


MyCnnModel(
  (conv1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=3, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=3, stride=2, padding=0)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (lstm1): LSTM(64, 64, batch_first=True)
  (lstm2): LSTM(64, 64, batch_first=True)
  (fc1): Linear(in_features=3840, out_features=32, bias=True)
  (activation): ReLU()
  (fc2): Linear(in_features=32, out_features=4, b

In [30]:
model.eval()

output= model(dev_data_X)
pre_lab = torch.argmax(output,dim=1)

# Convert predicted labels to {valence, activation} format using the label dictionary
pre_lab = pre_lab.tolist()


import json

# Define label dictionary for converting labels to {valence, activation} format

label_dict = {0: {"valence": 1, "activation": 1},
              1: {"valence": 1, "activation": 0},
              2: {"valence": 0, "activation": 0},
              3: {"valence": 0, "activation": 1}}

# Convert predicted labels to {valence, activation} format using the label dictionary

# Write predicted labels to a JSON file with line numbers and {valence, activation} format
with open("predicted_for_dev_0.85.json", "w") as file:
    data = {}
    for i, label in enumerate(pre_lab):
        data[str(i)] = label_dict[label]
    json.dump(data, file)

In [31]:
import pandas as pd
import numpy as np
import torch

test_file = 'test.json'
with open(test_file, 'r') as f:
         df=pd.read_json(f)
df=df.T
features = df["features"].values
lengths = [len(seq) for seq in features]
desired_seq_length = 500
padded_x = np.zeros((len(features), desired_seq_length, len(features[0][0])))
# loop through each sequence in x
for i, seq in enumerate(features):
    # check if sequence length is greater than desired length
      if len(seq) > desired_seq_length:
          padded_x[i] = seq[:desired_seq_length]
      else:
          padded_x[i, :len(seq), :] = seq
    # sequence length is already equal to desired length

test_data_X = torch.from_numpy(padded_x).float()

In [32]:
model.eval()

output= model(test_data_X)
pred_lab = torch.argmax(output,dim=1)

# Convert predicted labels to {valence, activation} format using the label dictionary
pred_lab = pred_lab.tolist()


import json

# Define label dictionary for converting labels to {valence, activation} format

label_dict = {0: {"valence": 1, "activation": 1},
              1: {"valence": 1, "activation": 0},
              2: {"valence": 0, "activation": 0},
              3: {"valence": 0, "activation": 1}}

# Convert predicted labels to {valence, activation} format using the label dictionary

# Write predicted labels to a JSON file with line numbers and {valence, activation} format
with open("prediction_test_0.85.json", "w") as file:
    data = {}
    for i, label in enumerate(pred_lab):
        data[str(i)] = label_dict[label]
    json.dump(data, file)